In [1]:
# This tells matplotlib not to try opening a new window for each plot.
%matplotlib inline

#make the depreciation warnings go away
import warnings
#I'm tired of the warnings on functions the professor asks us to use :) 
warnings.filterwarnings("ignore")

# General libraries.
import re
import numpy as np
import matplotlib.pyplot as plt
import nltk
import json
import pandas as pd

# SK-learn libraries for learning.
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.grid_search import GridSearchCV

# SK-learn libraries for evaluation.
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.model_selection import StratifiedShuffleSplit
# SK-learn libraries for feature extraction from text.
from sklearn.feature_extraction.text import *
import sys
from subprocess import check_output
print(sys.version)

 


3.5.2 |Anaconda custom (x86_64)| (default, Jul  2 2016, 17:52:12) 
[GCC 4.2.1 Compatible Apple LLVM 4.2 (clang-425.0.28)]


/Users/nicholeh/anaconda/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/nicholeh/anaconda/lib/python3.5/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
a = pd.read_json('train.json')
for x, y in enumerate(a.columns):
    print(x,y)

0 giver_username_if_known
1 number_of_downvotes_of_request_at_retrieval
2 number_of_upvotes_of_request_at_retrieval
3 post_was_edited
4 request_id
5 request_number_of_comments_at_retrieval
6 request_text
7 request_text_edit_aware
8 request_title
9 requester_account_age_in_days_at_request
10 requester_account_age_in_days_at_retrieval
11 requester_days_since_first_post_on_raop_at_request
12 requester_days_since_first_post_on_raop_at_retrieval
13 requester_number_of_comments_at_request
14 requester_number_of_comments_at_retrieval
15 requester_number_of_comments_in_raop_at_request
16 requester_number_of_comments_in_raop_at_retrieval
17 requester_number_of_posts_at_request
18 requester_number_of_posts_at_retrieval
19 requester_number_of_posts_on_raop_at_request
20 requester_number_of_posts_on_raop_at_retrieval
21 requester_number_of_subreddits_at_request
22 requester_received_pizza
23 requester_subreddits_at_request
24 requester_upvotes_minus_downvotes_at_request
25 requester_upvotes_minus_

In [3]:
#Nikki's first three hypotheses: date/time will matter

import datetime
def day_time(x):
    y = ''
    if datetime.datetime.fromtimestamp(x['unix_timestamp_of_request_utc']).hour < 10:
        y = 0
    elif datetime.datetime.fromtimestamp(x['unix_timestamp_of_request_utc']).hour >= 10 and datetime.datetime.fromtimestamp(x['unix_timestamp_of_request_utc']).hour < 16:
        y = 1
    elif datetime.datetime.fromtimestamp(x['unix_timestamp_of_request_utc']).hour >= 16 and datetime.datetime.fromtimestamp(x['unix_timestamp_of_request_utc']).hour < 21:
        y = 2
    else: 
        y = 3
    return y

day_values = ['morning', 'midday', 'evening', 'latenight']
    
def human_time(a):
    import datetime
    from datetime import date
    import calendar
    ### for the data in raop, return human time.  maybe the time of day matters
    a['human_readable_local_time'] = datetime.datetime.fromtimestamp(a['unix_timestamp_of_request']).strftime('%Y-%m-%d %H:%M:%S')
    a['human_readable_UTC_time'] = datetime.datetime.fromtimestamp(a['unix_timestamp_of_request_utc']).strftime('%Y-%m-%d %H:%M:%S')
    a['weekday'] = datetime.datetime.fromtimestamp(a['unix_timestamp_of_request']).weekday()
    a['month'] = datetime.datetime.fromtimestamp(a['unix_timestamp_of_request']).month
    a['time_of_day'] = day_time(a)
    return a


In [4]:
b = a.apply(human_time, axis=1)

In [5]:
#### add in number of downvotes.  The test data doesn't have this, it only has these aggregate values.  Derive number
### of downvotes
### we're given 
###up + down = col_26 
###and 
### up-down = col_24
### solve algebraically,
# 2up = col_26+col_24,
# up = (col_26+col_24)/2
# (col_26+col_24)/2 + down = col_26
#down = col_26 - (col_26+col_24)/2
#down = (col_26 - col_24)/2

b['total_downvotes'] = (b['requester_upvotes_plus_downvotes_at_request'] - b['requester_upvotes_minus_downvotes_at_request'])/2


In [6]:
def preprocess(x):
    """Use a series of regex expressions to remove unwanted characters"""
    #remove non-alpha-numeric characters, replace with whitespace
    x1 = re.sub(r'[^a-zA-Z_0-9_\s]'," ", x).lower()
    #replae all numbers with a single token and a space afterwards
    x1a = re.sub(r'[0-9]+', 'number ', x1)
    #x1b = re.sub(r'[_]+', ' ', x1a)
    #even though there are words that are just '_____', f1 actuall decreases when they're removed
    #remove newlines
    x2 = re.sub(r'[\n]', " ", x1a)
    #scrub out extra spaces
    x3 = re.sub(r'\s+', ' ', x2)  #other steps might have added extra space; remove
    return x3.strip()

In [7]:
#from http://scikit-learn.org/stable/auto_examples/hetero_feature_union.html
#try the Pipeline implementation from sk learn

import numpy as np
from sklearn.preprocessing import OneHotEncoder

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.datasets import fetch_20newsgroups
#from sklearn.datasets.twenty_newsgroups import strip_newsgroup_footer
#from sklearn.datasets.twenty_newsgroups import strip_newsgroup_quoting
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
from sklearn.pipeline import FeatureUnion
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import (RandomTreesEmbedding, RandomForestClassifier,
                              GradientBoostingClassifier)


class ItemSelector(BaseEstimator, TransformerMixin):
    """For data grouped by feature, select subset of data at a provided key.

    The data is expected to be stored in a 2D data structure, where the first
    index is over features and the second is over samples.  i.e.

    >> len(data[key]) == n_samples

    Please note that this is the opposite convention to scikit-learn feature
    matrixes (where the first index corresponds to sample).

    ItemSelector only requires that the collection implement getitem
    (data[key]).  Examples include: a dict of lists, 2D numpy array, Pandas
    DataFrame, numpy record array, etc.

    >> data = {'a': [1, 5, 2, 5, 2, 8],
               'b': [9, 4, 1, 4, 1, 3]}
    >> ds = ItemSelector(key='a')
    >> data['a'] == ds.transform(data)

    ItemSelector is not designed to handle data grouped by sample.  (e.g. a
    list of dicts).  If your data is structured this way, consider a
    transformer along the lines of `sklearn.feature_extraction.DictVectorizer`.

    Parameters
    ----------
    key : hashable, required
        The key corresponding to the desired value in a mappable.
    """
    def __init__(self, key):
        self.key = key

    def fit(self, x, y=None):
        return self

    def transform(self, data_dict):
        return data_dict[self.key]


class WeekdayExtractor(BaseEstimator, TransformerMixin):
    """Extract features from each document for DictVectorizer"""
    def __init__(self, key):
        self.key = key

    def fit(self, x, y=None):
        return self

    def transform(self, data_dict):
        shape = data_dict.shape
        return data_dict[self.key].reshape(shape[0],1)

class SubjectBodyExtractor(BaseEstimator, TransformerMixin):
    """Extract the subject & body from a usenet post in a single pass.

    Takes a sequence of strings and produces a dict of sequences.  Keys are
    `subject` and `body`.
    """
    def fit(self, x, y=None):
        return self

    def transform(self, posts):
        features = np.recarray(shape=(len(posts),),
                               dtype=[('subject', object), ('body', object)])
        for i, text in enumerate(posts):
            headers, _, bod = text.partition('\n\n')
            bod = strip_newsgroup_footer(bod)
            bod = strip_newsgroup_quoting(bod)
            features['body'][i] = bod

            prefix = 'Subject:'
            sub = ''
            for line in headers.split('\n'):
                if line.startswith(prefix):
                    sub = line[len(prefix):]
                    break
            features['subject'][i] = sub

        return features

class NBWrapper(BaseEstimator, TransformerMixin):
    """Extract features from each document for DictVectorizer"""
    def __init__(self, NB):
        self.NB = NB

    def fit(self, x, y=None):
        self.NB.fit(x, y)
        return self

    def transform(self, posts):
        return self.NB.predict(posts)

pipeline4= Pipeline([
    # Extract the subject & body
    #('subjectbody', SubjectBodyExtractor()),

    # Use FeatureUnion to combine the features from subject and body
    ('union', FeatureUnion(
        transformer_list=[
        ### add in text variables
            # Pipeline for pulling features from the post's subject line
            ('get-title', Pipeline([
                ('selector', ItemSelector(key='request_title')),
           #     ('cv', CountVectorizer(analyzer='char_wb', 
           #                           #vocabulary=vocabulary,
           #                            #preprocessor=preprocess,
           #                          max_df=0.5, ngram_range=(1,2))),
                ('tfidf', TfidfVectorizer(min_df=65, 
                                          preprocessor=preprocess
                                         ))
            ])),

            
            #('get-text', Pipeline([
            #    ('selector', ItemSelector(key='request_text_edit_aware')),
            #    ('tfidf', TfidfVectorizer(min_df=50))
                #('cv', CountVectorizer(analyzer='char_wb', 
                #                       #vocabulary=vocabulary,
                #                       max_df=0.5, ngram_range=(1,3)))
            #])),
         ### adding in categorical variables
           # research different encoders
           ('get-time_of_day', Pipeline([
                ('selector', WeekdayExtractor(key='time_of_day')),
                ('one_hot', OneHotEncoder())
                #('NB', NBWrapper(MultinomialNB()))             
           ])),
        
           ('get-weekday', Pipeline([
                ('selector', WeekdayExtractor(key='weekday')),
                ('lh', OneHotEncoder())
                #('NB', NBWrapper(MultinomialNB()))             
           ])),
                    
          ('get-month', Pipeline([
                ('selector', WeekdayExtractor(key='month')),
                ('2h', OneHotEncoder())
                #('NB', NBWrapper(MultinomialNB()))             
           ])),
                    
                    
        ### add numeric data
        #research better numeric extractors and coders
         ('get-requester_number_of_subreddits_at_request', Pipeline([
                ('selector', WeekdayExtractor(key='requester_number_of_subreddits_at_request')),
                #('lb', OneHotEncoder())
                #('NB', NBWrapper(MultinomialNB()))             
           ])),
            
         ('get-downvotes', Pipeline([
                ('selector', WeekdayExtractor(key='total_downvotes')),
                #('lb', OneHotEncoder())
                #('NB', NBWrapper(MultinomialNB()))   
                
           ]))
        ]
    )),

    # Use a SVC classifier on the combined features
    #('estimator',LogisticRegression(penalty='l2',C=25))
    #('estimator', SVC(kernel='linear')) 
    
    ('estimator', DecisionTreeClassifier())
    #('estimator', RandomForestClassifier())
    #('estimator', MultinomialNB())
    #('estmator', KNeighborsClassifier())
])



In [8]:
data_set = b[['request_title','requester_number_of_subreddits_at_request', 
              'time_of_day','weekday', 'month', 'total_downvotes']]
#data_set_title = a['request_title']
data_labels = b['requester_received_pizza']


sss = StratifiedShuffleSplit(test_size=0.25, random_state=1)
for train_index, test_index in sss.split(data_set, data_labels):
    X_train, X_test = data_set.iloc[train_index], data_set.iloc[test_index]
    y_train, y_test = data_labels.iloc[train_index], data_labels.iloc[test_index]

pipeline4.fit(X_train, y_train)
y_pred = pipeline4.predict(X_test)
target_names = ['No pizza','Got Pizza']
print(classification_report(y_test, y_pred, target_names=target_names))

             precision    recall  f1-score   support

   No pizza       0.78      0.78      0.78       762
  Got Pizza       0.31      0.31      0.31       248

avg / total       0.66      0.66      0.66      1010



In [9]:
confusion_matrix(y_pred, y_test)

array([[594, 172],
       [168,  76]])

In [11]:
test = pd.read_json('test.json')
test['total_downvotes'] = (test['requester_upvotes_plus_downvotes_at_request'] - test['requester_upvotes_minus_downvotes_at_request'])/2

In [13]:
test1 = test.apply(human_time, axis=1)

In [14]:
test1.head()

,giver_username_if_known,request_id,request_text_edit_aware,request_title,requester_account_age_in_days_at_request,requester_days_since_first_post_on_raop_at_request,requester_number_of_comments_at_request,requester_number_of_comments_in_raop_at_request,requester_number_of_posts_at_request,requester_number_of_posts_on_raop_at_request,...,requester_upvotes_plus_downvotes_at_request,requester_username,unix_timestamp_of_request,unix_timestamp_of_request_utc,total_downvotes,human_readable_local_time,human_readable_UTC_time,weekday,month,time_of_day
0,N/A,t3_i8iy4,Hey all! It's about 95 degrees here and our ki...,[request] pregger gf 95 degree house and no fo...,42.083866,0.0,57,0,10,0,...,840,j_like,1308963419,1308959819,238.0,2011-06-24 17:56:59,2011-06-24 16:56:59,4,6,2
1,N/A,t3_1mfqi0,I didn't know a place like this exists! \n\nI ...,"[Request] Lost my job day after labour day, st...",223.784537,0.0,145,0,36,0,...,1448,0110110101101100,1379263523,1379259923,466.0,2013-09-15 09:45:23,2013-09-15 08:45:23,6,9,0
2,N/A,t3_lclka,Hi Reddit. Im a single dad having a really rou...,(Request) pizza for my kids please?,0.000000,0.0,0,0,0,0,...,0,singledad22601,1318636421,1318632821,0.0,2011-10-14 16:53:41,2011-10-14 15:53:41,4,10,1
3,N/A,t3_1jdgdj,Hi I just moved to Waltham MA from my home sta...,[Request] Just moved to a new state(Waltham MA...,481.311273,0.0,277,0,17,0,...,2062,Neuronut,1375220282,1375216682,502.0,2013-07-30 14:38:02,2013-07-30 13:38:02,1,7,1
4,N/A,t3_t2qt4,We're just sitting here near indianapolis on o...,"[Request] Two girls in between paychecks, we'v...",0.000000,0.0,0,0,0,0,...,0,so_damn_hungry,1335934358,1335930758,0.0,2012-05-01 21:52:38,2012-05-01 20:52:38,1,5,2


In [15]:
test_set = test1[['request_title','requester_number_of_subreddits_at_request', 
              'time_of_day','weekday', 'month', 'total_downvotes']]

In [16]:
test_set.head()

,request_title,requester_number_of_subreddits_at_request,time_of_day,weekday,month,total_downvotes
0,[request] pregger gf 95 degree house and no fo...,16,2,4,6,238.0
1,"[Request] Lost my job day after labour day, st...",29,0,6,9,466.0
2,(Request) pizza for my kids please?,0,1,4,10,0.0
3,[Request] Just moved to a new state(Waltham MA...,30,1,1,7,502.0
4,"[Request] Two girls in between paychecks, we'v...",0,2,1,5,0.0


In [17]:
test_pred = pipeline4.predict(test_set)

In [18]:
np.average(test_pred)

0.25015328019619865

In [21]:
def inter(x):
    return int(x)
vint = np.vectorize(inter)
test_pred2 = vint(test_pred)
fin_df = pd.concat([test1['request_id'],pd.DataFrame(test_pred2, columns=['requester_received_pizza'])],axis =1)
fin_df['requester_received_pizza'] = pd.to_numeric(fin_df['requester_received_pizza'],downcast='signed')
fin_df.head()

,request_id,requester_received_pizza
0,t3_i8iy4,1
1,t3_1mfqi0,0
2,t3_lclka,0
3,t3_1jdgdj,0
4,t3_t2qt4,1


In [22]:
fin_df.to_csv('pizza_submission13.csv', sep = ',', index=False, header=True)

In [25]:
#from http://scikit-learn.org/stable/auto_examples/hetero_feature_union.html
#try the Pipeline implementation from sk learn

import numpy as np
from sklearn.preprocessing import OneHotEncoder

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.datasets import fetch_20newsgroups
#from sklearn.datasets.twenty_newsgroups import strip_newsgroup_footer
#from sklearn.datasets.twenty_newsgroups import strip_newsgroup_quoting
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
from sklearn.pipeline import FeatureUnion
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import (RandomTreesEmbedding, RandomForestClassifier,
                              GradientBoostingClassifier)
from sklearn.ensemble import AdaBoostClassifier

class ItemSelector(BaseEstimator, TransformerMixin):
    """For data grouped by feature, select subset of data at a provided key.

    The data is expected to be stored in a 2D data structure, where the first
    index is over features and the second is over samples.  i.e.

    >> len(data[key]) == n_samples

    Please note that this is the opposite convention to scikit-learn feature
    matrixes (where the first index corresponds to sample).

    ItemSelector only requires that the collection implement getitem
    (data[key]).  Examples include: a dict of lists, 2D numpy array, Pandas
    DataFrame, numpy record array, etc.

    >> data = {'a': [1, 5, 2, 5, 2, 8],
               'b': [9, 4, 1, 4, 1, 3]}
    >> ds = ItemSelector(key='a')
    >> data['a'] == ds.transform(data)

    ItemSelector is not designed to handle data grouped by sample.  (e.g. a
    list of dicts).  If your data is structured this way, consider a
    transformer along the lines of `sklearn.feature_extraction.DictVectorizer`.

    Parameters
    ----------
    key : hashable, required
        The key corresponding to the desired value in a mappable.
    """
    def __init__(self, key):
        self.key = key

    def fit(self, x, y=None):
        return self

    def transform(self, data_dict):
        return data_dict[self.key]


class WeekdayExtractor(BaseEstimator, TransformerMixin):
    """Extract features from each document for DictVectorizer"""
    def __init__(self, key):
        self.key = key

    def fit(self, x, y=None):
        return self

    def transform(self, data_dict):
        shape = data_dict.shape
        return data_dict[self.key].reshape(shape[0],1)

class SubjectBodyExtractor(BaseEstimator, TransformerMixin):
    """Extract the subject & body from a usenet post in a single pass.

    Takes a sequence of strings and produces a dict of sequences.  Keys are
    `subject` and `body`.
    """
    def fit(self, x, y=None):
        return self

    def transform(self, posts):
        features = np.recarray(shape=(len(posts),),
                               dtype=[('subject', object), ('body', object)])
        for i, text in enumerate(posts):
            headers, _, bod = text.partition('\n\n')
            bod = strip_newsgroup_footer(bod)
            bod = strip_newsgroup_quoting(bod)
            features['body'][i] = bod

            prefix = 'Subject:'
            sub = ''
            for line in headers.split('\n'):
                if line.startswith(prefix):
                    sub = line[len(prefix):]
                    break
            features['subject'][i] = sub

        return features

class NBWrapper(BaseEstimator, TransformerMixin):
    """Extract features from each document for DictVectorizer"""
    def __init__(self, NB):
        self.NB = NB

    def fit(self, x, y=None):
        self.NB.fit(x, y)
        return self

    def transform(self, posts):
        return self.NB.predict(posts)

pipeline4= Pipeline([
    # Extract the subject & body
    #('subjectbody', SubjectBodyExtractor()),

    # Use FeatureUnion to combine the features from subject and body
    ('union', FeatureUnion(
        transformer_list=[
        ### add in text variables
            # Pipeline for pulling features from the post's subject line
            ('get-title', Pipeline([
                ('selector', ItemSelector(key='request_title')),
           #     ('cv', CountVectorizer(analyzer='char_wb', 
           #                           #vocabulary=vocabulary,
           #                            #preprocessor=preprocess,
           #                          max_df=0.5, ngram_range=(1,2))),
                ('tfidf', TfidfVectorizer(min_df=65, 
                                          preprocessor=preprocess
                                         ))
            ])),

            
            #('get-text', Pipeline([
            #    ('selector', ItemSelector(key='request_text_edit_aware')),
            #    ('tfidf', TfidfVectorizer(min_df=50))
                #('cv', CountVectorizer(analyzer='char_wb', 
                #                       #vocabulary=vocabulary,
                #                       max_df=0.5, ngram_range=(1,3)))
            #])),
         ### adding in categorical variables
           # research different encoders
           ('get-time_of_day', Pipeline([
                ('selector', WeekdayExtractor(key='time_of_day')),
                ('one_hot', OneHotEncoder())
                #('NB', NBWrapper(MultinomialNB()))             
           ])),
        
           ('get-weekday', Pipeline([
                ('selector', WeekdayExtractor(key='weekday')),
                ('lh', OneHotEncoder())
                #('NB', NBWrapper(MultinomialNB()))             
           ])),
                    
          ('get-month', Pipeline([
                ('selector', WeekdayExtractor(key='month')),
                ('2h', OneHotEncoder())
                #('NB', NBWrapper(MultinomialNB()))             
           ])),
                    
                    
        ### add numeric data
        #research better numeric extractors and coders
         ('get-requester_number_of_subreddits_at_request', Pipeline([
                ('selector', WeekdayExtractor(key='requester_number_of_subreddits_at_request')),
                #('lb', OneHotEncoder())
                #('NB', NBWrapper(MultinomialNB()))             
           ])),
            
         ('get-downvotes', Pipeline([
                ('selector', WeekdayExtractor(key='total_downvotes')),
                #('lb', OneHotEncoder())
                #('NB', NBWrapper(MultinomialNB()))   
                
           ]))
        ]
    )),

    # Use a SVC classifier on the combined features
    #('estimator',LogisticRegression(penalty='l2',C=25))
    #('estimator', SVC(kernel='linear')) 
    
    ('estimator', AdaBoostClassifier(RandomForestClassifier()))
    #('estimator', RandomForestClassifier())
    #('estimator', MultinomialNB())
    #('estmator', KNeighborsClassifier())
])



In [26]:
data_set = b[['request_title','requester_number_of_subreddits_at_request', 
              'time_of_day','weekday', 'month', 'total_downvotes']]



#data_set_title = a['request_title']
data_labels = b['requester_received_pizza']


sss = StratifiedShuffleSplit(test_size=0.25, random_state=1)
for train_index, test_index in sss.split(data_set, data_labels):
    X_train, X_test = data_set.iloc[train_index], data_set.iloc[test_index]
    y_train, y_test = data_labels.iloc[train_index], data_labels.iloc[test_index]

pipeline4.fit(X_train, y_train)
y_pred = pipeline4.predict(X_test)
target_names = ['No pizza','Got Pizza']
print(classification_report(y_test, y_pred, target_names=target_names))

             precision    recall  f1-score   support

   No pizza       0.76      0.95      0.84       762
  Got Pizza       0.30      0.07      0.11       248

avg / total       0.64      0.73      0.66      1010



In [30]:
from sklearn.preprocessing import MaxAbsScaler
b['request_length'] = b['request_text_edit_aware'].map(lambda x: len(x.split()))


cols = ['requester_account_age_in_days_at_request',
       'requester_days_since_first_post_on_raop_at_request',
       'requester_number_of_comments_at_request',
       'requester_number_of_comments_in_raop_at_request',
       'requester_number_of_posts_at_request',
       'requester_number_of_posts_on_raop_at_request',
       'requester_number_of_subreddits_at_request','requester_upvotes_minus_downvotes_at_request',
       'requester_upvotes_plus_downvotes_at_request',
       'request_length',
       'total_downvotes'
       ]
scl = MaxAbsScaler()
num_norm = pd.DataFrame(index=b.index)
for col in cols:
    scl.fit(b[col].values.reshape(-1,1))
    c = col+'_scaled'
    num_norm[c] = scl.transform(b[col].values.reshape(-1, 1))

In [39]:
 #from http://scikit-learn.org/stable/auto_examples/hetero_feature_union.html
#try the Pipeline implementation from sk learn

import numpy as np
from sklearn.preprocessing import OneHotEncoder

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.datasets import fetch_20newsgroups
#from sklearn.datasets.twenty_newsgroups import strip_newsgroup_footer
#from sklearn.datasets.twenty_newsgroups import strip_newsgroup_quoting
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
from sklearn.pipeline import FeatureUnion
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import (RandomTreesEmbedding, RandomForestClassifier,
                              GradientBoostingClassifier)
from sklearn.ensemble import AdaBoostClassifier

class ItemSelector(BaseEstimator, TransformerMixin):
    """For data grouped by feature, select subset of data at a provided key.

    The data is expected to be stored in a 2D data structure, where the first
    index is over features and the second is over samples.  i.e.

    >> len(data[key]) == n_samples

    Please note that this is the opposite convention to scikit-learn feature
    matrixes (where the first index corresponds to sample).

    ItemSelector only requires that the collection implement getitem
    (data[key]).  Examples include: a dict of lists, 2D numpy array, Pandas
    DataFrame, numpy record array, etc.

    >> data = {'a': [1, 5, 2, 5, 2, 8],
               'b': [9, 4, 1, 4, 1, 3]}
    >> ds = ItemSelector(key='a')
    >> data['a'] == ds.transform(data)

    ItemSelector is not designed to handle data grouped by sample.  (e.g. a
    list of dicts).  If your data is structured this way, consider a
    transformer along the lines of `sklearn.feature_extraction.DictVectorizer`.

    Parameters
    ----------
    key : hashable, required
        The key corresponding to the desired value in a mappable.
    """
    def __init__(self, key):
        self.key = key

    def fit(self, x, y=None):
        return self

    def transform(self, data_dict):
        return data_dict[self.key]


class WeekdayExtractor(BaseEstimator, TransformerMixin):
    """Extract features from each document for DictVectorizer"""
    def __init__(self, key):
        self.key = key

    def fit(self, x, y=None):
        return self

    def transform(self, data_dict):
        shape = data_dict.shape
        return data_dict[self.key].reshape(shape[0],1)

class SubjectBodyExtractor(BaseEstimator, TransformerMixin):
    """Extract the subject & body from a usenet post in a single pass.

    Takes a sequence of strings and produces a dict of sequences.  Keys are
    `subject` and `body`.
    """
    def fit(self, x, y=None):
        return self

    def transform(self, posts):
        features = np.recarray(shape=(len(posts),),
                               dtype=[('subject', object), ('body', object)])
        for i, text in enumerate(posts):
            headers, _, bod = text.partition('\n\n')
            bod = strip_newsgroup_footer(bod)
            bod = strip_newsgroup_quoting(bod)
            features['body'][i] = bod

            prefix = 'Subject:'
            sub = ''
            for line in headers.split('\n'):
                if line.startswith(prefix):
                    sub = line[len(prefix):]
                    break
            features['subject'][i] = sub

        return features

class NBWrapper(BaseEstimator, TransformerMixin):
    """Extract features from each document for DictVectorizer"""
    def __init__(self, NB):
        self.NB = NB

    def fit(self, x, y=None):
        self.NB.fit(x, y)
        return self

    def transform(self, posts):
        return self.NB.predict(posts)

pipeline4= Pipeline([
    # Extract the subject & body
    #('subjectbody', SubjectBodyExtractor()),

    # Use FeatureUnion to combine the features from subject and body
    ('union', FeatureUnion(
        transformer_list=[
        ### add in text variables
            # Pipeline for pulling features from the post's subject line
            ('get-title', Pipeline([
                ('selector', ItemSelector(key='request_title')),
           #     ('cv', CountVectorizer(analyzer='char_wb', 
           #                           #vocabulary=vocabulary,
           #                            #preprocessor=preprocess,
           #                          max_df=0.5, ngram_range=(1,2))),
                ('tfidf', TfidfVectorizer(min_df=65, 
                                          preprocessor=preprocess
                                         ))
            ])),

            
            #('get-text', Pipeline([
            #    ('selector', ItemSelector(key='request_text_edit_aware')),
            #    ('tfidf', TfidfVectorizer(min_df=50))
                #('cv', CountVectorizer(analyzer='char_wb', 
                #                       #vocabulary=vocabulary,
                #                       max_df=0.5, ngram_range=(1,3)))
            #])),
         ### adding in categorical variables
           # research different encoders
           ('get-time_of_day', Pipeline([
                ('selector', WeekdayExtractor(key='time_of_day')),
                ('one_hot', OneHotEncoder())
                #('NB', NBWrapper(MultinomialNB()))             
           ])),
        
           ('get-weekday', Pipeline([
                ('selector', WeekdayExtractor(key='weekday')),
                ('lh', OneHotEncoder())
                #('NB', NBWrapper(MultinomialNB()))             
           ])),
                    
          ('get-month', Pipeline([
                ('selector', WeekdayExtractor(key='month')),
                ('2h', OneHotEncoder())
                #('NB', NBWrapper(MultinomialNB()))             
           ])),
                    
            ('get-requester_number_of_subreddits_at_request', Pipeline([
                ('selector', WeekdayExtractor(key='requester_number_of_subreddits_at_request_scaled')),
                #('lb', OneHotEncoder())
                #('NB', NBWrapper(MultinomialNB()))             
           ])),        
        ### add numeric data
        #research better numeric extractors and coders
         ('get-requester_account_age_in_days_at_request_scaled', Pipeline([
                ('selector', WeekdayExtractor(key='requester_account_age_in_days_at_request_scaled')),
                #('lb', OneHotEncoder())
                #('NB', NBWrapper(MultinomialNB()))             
           ])),
          
                    
        ('get-requester_days_since_first_post_on_raop_at_request_scaled', Pipeline([
                ('selector', WeekdayExtractor(key='requester_days_since_first_post_on_raop_at_request_scaled')),
                #('lb', OneHotEncoder())
                #('NB', NBWrapper(MultinomialNB()))             
        ])),
        #('get-requester_number_of_comments_at_request_scaled', Pipeline([
        #        ('selector', WeekdayExtractor(key='requester_number_of_comments_at_request_scaled')),
        #        #('lb', OneHotEncoder())
        #        #('NB', NBWrapper(MultinomialNB()))             
        #])),    
        #('get-requester_number_of_comments_in_raop_at_request_scaled', Pipeline([
        #        ('selector', WeekdayExtractor(key='requester_number_of_comments_in_raop_at_request_scaled')),
        #        #('lb', OneHotEncoder())
        #        #('NB', NBWrapper(MultinomialNB()))             
        #])),
        #('get-requester_number_of_posts_at_request_scaled', Pipeline([
        #        ('selector', WeekdayExtractor(key='requester_number_of_posts_at_request_scaled')),
        #        #('lb', OneHotEncoder())
        #        #('NB', NBWrapper(MultinomialNB()))             
        #])),
        #('get-requester_number_of_posts_on_raop_at_request_scaled', Pipeline([
        #        ('selector', WeekdayExtractor(key='requester_number_of_posts_on_raop_at_request_scaled')),
        #        #('lb', OneHotEncoder())
        #        #('NB', NBWrapper(MultinomialNB()))             
        #])), 
        ('get-requester_number_of_subreddits_at_request_scaled', Pipeline([
                ('selector', WeekdayExtractor(key='requester_number_of_subreddits_at_request_scaled')),
                #('lb', OneHotEncoder())
                #('NB', NBWrapper(MultinomialNB()))             
        ])),                     
         ('get-downvotes', Pipeline([
                ('selector', WeekdayExtractor(key='total_downvotes_scaled')),
                #('lb', OneHotEncoder())
                #('NB', NBWrapper(MultinomialNB()))   
                
           ]))
        ]
    )),

    # Use a SVC classifier on the combined features
    #('estimator',LogisticRegression(penalty='l2',C=25))
    #('estimator', SVC(kernel='linear')) 
    
    ('estimator', AdaBoostClassifier(RandomForestClassifier()))
    #('estimator', RandomForestClassifier())
    #('estimator', MultinomialNB())
    #('estmator', KNeighborsClassifier())
])



In [40]:
data_set = pd.concat([b[['request_title', 
              'time_of_day','weekday', 'month']],num_norm[['requester_account_age_in_days_at_request_scaled',
'requester_days_since_first_post_on_raop_at_request_scaled',
#'requester_number_of_comments_in_raop_at_request_scaled',
#'requester_number_of_posts_on_raop_at_request_scaled',
'requester_number_of_subreddits_at_request_scaled',
'request_length_scaled',
'total_downvotes_scaled']]], axis = 1)
#data_set_title = a['request_title']
data_labels = b['requester_received_pizza']


sss = StratifiedShuffleSplit(test_size=0.25, random_state=1)
for train_index, test_index in sss.split(data_set, data_labels):
    X_train, X_test = data_set.iloc[train_index], data_set.iloc[test_index]
    y_train, y_test = data_labels.iloc[train_index], data_labels.iloc[test_index]

pipeline4.fit(X_train, y_train)
y_pred = pipeline4.predict(X_test)
target_names = ['No pizza','Got Pizza']
print(classification_report(y_test, y_pred, target_names=target_names))

             precision    recall  f1-score   support

   No pizza       0.76      0.95      0.84       762
  Got Pizza       0.27      0.06      0.09       248

avg / total       0.64      0.73      0.66      1010



In [33]:
num_norm.columns

Index(['requester_account_age_in_days_at_request_scaled',
       'requester_days_since_first_post_on_raop_at_request_scaled',
       'requester_number_of_comments_at_request_scaled',
       'requester_number_of_comments_in_raop_at_request_scaled',
       'requester_number_of_posts_at_request_scaled',
       'requester_number_of_posts_on_raop_at_request_scaled',
       'requester_number_of_subreddits_at_request_scaled',
       'requester_upvotes_minus_downvotes_at_request_scaled',
       'requester_upvotes_plus_downvotes_at_request_scaled',
       'request_length_scaled', 'total_downvotes_scaled'],
      dtype='object')

In [41]:
b.head()

,giver_username_if_known,number_of_downvotes_of_request_at_retrieval,number_of_upvotes_of_request_at_retrieval,post_was_edited,request_id,request_number_of_comments_at_retrieval,request_text,request_text_edit_aware,request_title,requester_account_age_in_days_at_request,...,requester_username,unix_timestamp_of_request,unix_timestamp_of_request_utc,human_readable_local_time,human_readable_UTC_time,weekday,month,time_of_day,total_downvotes,request_length
0,N/A,0,1,0,t3_l25d7,0,Hi I am in need of food for my 4 children we a...,Hi I am in need of food for my 4 children we a...,Request Colorado Springs Help Us Please,0.000000,...,nickylvst,1317852607,1317849007,2011-10-05 15:10:07,2011-10-05 14:10:07,2,10,1,0.0,67
1,N/A,2,5,0,t3_rcb83,0,I spent the last money I had on gas today. Im ...,I spent the last money I had on gas today. Im ...,"[Request] California, No cash and I could use ...",501.111100,...,fohacidal,1332652424,1332648824,2012-03-24 22:13:44,2012-03-24 21:13:44,5,3,3,41.0,16
2,N/A,0,3,0,t3_lpu5j,0,My girlfriend decided it would be a good idea ...,My girlfriend decided it would be a good idea ...,"[Request] Hungry couple in Dundee, Scotland wo...",0.000000,...,jacquibatman7,1319650094,1319646494,2011-10-26 10:28:14,2011-10-26 09:28:14,2,10,0,0.0,59
3,N/A,0,1,1,t3_mxvj3,4,"It's cold, I'n hungry, and to be completely ho...","It's cold, I'n hungry, and to be completely ho...","[Request] In Canada (Ontario), just got home f...",6.518438,...,4on_the_floor,1322855434,1322855434,2011-12-02 11:50:34,2011-12-02 11:50:34,4,12,1,11.0,30
4,N/A,6,6,0,t3_1i6486,5,hey guys:\n I love this sub. I think it's grea...,hey guys:\n I love this sub. I think it's grea...,[Request] Old friend coming to visit. Would LO...,162.063252,...,Futuredogwalker,1373657691,1373654091,2013-07-12 12:34:51,2013-07-12 11:34:51,4,7,1,306.0,103


In [46]:
print('successful downvotes average:', np.average(b[b['requester_received_pizza'] == True]['total_downvotes']))
print('successful downvotes stddev:', np.std(b[b['requester_received_pizza'] == True]['total_downvotes']))
print('unsuccessful downvotes average:', np.average(b[b['requester_received_pizza'] == False]['total_downvotes']))
print('unsuccessful downvotes stddev:', np.std(b[b['requester_received_pizza'] == False]['total_downvotes']))

successful downvotes average: 1775.81991952
successful downvotes stddev: 18470.728594117507
unsuccessful downvotes average: 1133.5554826
unsuccessful downvotes stddev: 7490.887224296757


1133.5554826001314